In [1]:
%matplotlib inline
import re
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import numpy
import torch
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
import keras

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train.csv",header=None)

In [3]:
test = pd.read_csv("test.csv",header=None)

In [ ]:
train.shape

In [4]:
trainVals = train.values
testVals = test.values

In [5]:
X_test = testVals
X_test = np.delete(X_test,0,axis=1)

In [6]:
Y_train = trainVals[:,len(trainVals[0])-1]
X_train = trainVals
X_train=np.delete(X_train, 88200, axis=1)

In [ ]:
trainFFT = np.zeros(X_train.shape)
print(len(X_train))
for i in range(len(X_train)):
    trainFFT[i] = np.fft.fft(X_train[i])

In [49]:
#extracting mfcc means
trainMFCCs = np.zeros((X_train.shape[0],173,10))
for i in range(len(X_train)):
    trainMFCCs[i] = librosa.feature.melspectrogram(y=X_train[i],sr = 44100,n_mels=10,hop_length = 512).T

88200

In [ ]:
print(trainMFCCs.shape)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
bagging = BaggingClassifier(SVC(C=1,gamma = .5),
                            max_samples=.75, max_features=.75, n_estimators=50)
bagging.fit(trainMFCCs,Y_train)

In [ ]:
print(accuracy_score(Y_train,bagging.predict(trainMFCCs)))

In [57]:
testMFCCs = np.zeros((X_test.shape[0],173,10))
for i in range(len(X_test)):
    testMFCCs[i] = librosa.feature.melspectrogram(y=X_test[i],sr = 44100,n_mels=10,hop_length=512).T
testFlat = []
for i in range(testMFCCs.shape[0]):
    temp = []
    for x in range(testMFCCs.shape[1]):
        for z in range(testMFCCs.shape[2]):
            temp.append(testMFCCs[i][x][z])
    testFlat.append(temp)

In [ ]:
print(trainMFCCs.shape)

In [34]:
flatTrain = []
for i in range(trainMFCCs.shape[0]):
    temp = []
    for x in range(trainMFCCs.shape[1]):
        for z in range(trainMFCCs.shape[2]):
            temp.append(trainMFCCs[i][x][z])
    flatTrain.append(temp)
print(trainMFCCs.shape)
print(len(flatTrain[0]))

(6325, 690, 40)
27600


In [35]:
Yval = Y_train[:200]
flatTrain = np.array(flatTrain)
Xval = flatTrain[:200]
model = Sequential()
model.add(Dense(2056, input_shape=(len(flatTrain[1]),), activation = 'relu'))
#model.add(Dropout(0.1))
model.add(Dense(2056, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(2056, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(2056, activation='relu'))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(flatTrain[200:],Y_train[200:],epochs = 15,batch_size=32,validation_data = (Xval,Yval))

Train on 6125 samples, validate on 200 samples
Epoch 1/15
6125/6125 [==============================] - 100s 16ms/step - loss: 3.0690 - acc: 0.5838 - val_loss: 2.3898 - val_acc: 0.6700
Epoch 2/15
6125/6125 [==============================] - 99s 16ms/step - loss: 2.0063 - acc: 0.6731 - val_loss: 3.4468 - val_acc: 0.5400
Epoch 3/15
6125/6125 [==============================] - 116s 19ms/step - loss: 1.5570 - acc: 0.7409 - val_loss: 2.2065 - val_acc: 0.6950
Epoch 4/15
6125/6125 [==============================] - 119s 19ms/step - loss: 1.0159 - acc: 0.8300 - val_loss: 1.3588 - val_acc: 0.8350
Epoch 5/15
6125/6125 [==============================] - 118s 19ms/step - loss: 0.8864 - acc: 0.8444 - val_loss: 1.1480 - val_acc: 0.8050
Epoch 6/15
6125/6125 [==============================] - 117s 19ms/step - loss: 0.8212 - acc: 0.8684 - val_loss: 1.1075 - val_acc: 0.8400
Epoch 7/15
6125/6125 [==============================] - 118s 19ms/step - loss: 0.6667 - acc: 0.8877 - val_loss: 1.4013 - val_acc: 0.

In [77]:
'''X_trainLSTM = np.zeros((X_train.shape[0],X_train.shape[1],1))
for i in range(X_train.shape[0]):
    for x in range(X_train.shape[1]):
        X_trainLSTM[i][x][0] = X_train[i][x]'''
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
#model.add(LSTM(1,input_shape=(X_trainLSTM.shape[1],X_trainLSTM.shape[2])))
#model.add(LSTM(256, input_shape=(trainMFCCs.shape[1],trainMFCCs.shape[2])))
#model.add(LSTM(256))
#model.add(Dense(units=512))
model.add(Dense(units=10, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.fit(X_train[200:],Y_train[200:],epochs = 5,batch_size=32,validation_data = (X_train[:200],Y_train[:200]))

Train on 6125 samples, validate on 200 samples
Epoch 1/5
6125/6125 [==============================] - 57s 9ms/step - loss: 0.8743 - acc: 0.9154 - val_loss: 0.1770 - val_acc: 1.0000
Epoch 2/5
6125/6125 [==============================] - 56s 9ms/step - loss: 0.1021 - acc: 0.9962 - val_loss: 0.0610 - val_acc: 1.0000
Epoch 3/5
6125/6125 [==============================] - 55s 9ms/step - loss: 0.0425 - acc: 0.9989 - val_loss: 0.0325 - val_acc: 1.0000
Epoch 4/5
6125/6125 [==============================] - 55s 9ms/step - loss: 0.0236 - acc: 0.9998 - val_loss: 0.0212 - val_acc: 1.0000
Epoch 5/5
6125/6125 [==============================] - 55s 9ms/step - loss: 0.0159 - acc: 0.9998 - val_loss: 0.0155 - val_acc: 1.0000


In [80]:
preds1 = model.predict(X_test)
preds = [np.argmax(i) for i in preds1]

In [30]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i) + "," + str(p) + "\n")

In [81]:
write_to_file("predictionsRawDense256Dropout02Dense256.csv",preds)